In [1]:
sc

In [2]:
import datetime
import operator
import os
import sys
import time
import pyspark
from operator import add
from math import radians, cos, sin, asin, sqrt
start = time.time()

In [3]:
def indexZones(shapeFilename):
    import rtree
    import fiona.crs
    import geopandas as gpd
    index = rtree.Rtree()
    zones = gpd.read_file(shapeFilename).to_crs(fiona.crs.from_epsg(2263))
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def find_Neighborhood(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if any(map(lambda x: x.contains(p), zones.geometry[idx])):
            return zones['neighborhood'][idx]
    return -1

In [4]:
def mapToZone(parts):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
    index, zones = indexZones('neighborhoods.geojson')
    
    for line in parts[1:]:
        fields = line.strip('').split(',')
        if fields ==['']: continue
        if all((fields[5],fields[6],fields[9],fields[10])):
            for i in [5,6,9,10]:
                fields[i] = fields[i].strip('').split("\"")[1]
            pickup_location  = geom.Point(proj(float(fields[6]), float(fields[5])))
            dropoff_location = geom.Point(proj(float(fields[10]), float(fields[9])))
            pickup_zone = find_Neighborhood(pickup_location, index, zones)
            dropoff_zone = find_Neighborhood(dropoff_location, index, zones)

            if pickup_zone>=0 and dropoff_zone>=0:
                yield ((dropoff_zone, pickup_zone), 1)

In [8]:

trips  =  sc.textFile('../CitiBike.csv')
output = trips.mapPartitions(mapToZone)
output = mapToZone(trips.take(100))
output1 = sc.parallelize(output)
print (time.time()-start)/60.0

3.09041475058
